# **embedding과 rnn**

## 1. 이론 정리

### (1) 임베딩(Embedding)이란?
![[image/Pasted image 20251022131229.png]]
> 단어를 실수 벡터로 변환하는 학습 가능한 레이어

|구분|원-핫 인코딩|임베딩|
|---|---|---|
|벡터 형태|[0, 0, 1, 0, 0, ...]|[0.12, -0.45, 0.87, ...]|
|차원|어휘 수만큼 (희소)|저차원 (밀집)|
|의미 반영|X|✅ 유사 단어끼리 벡터가 가까움|

**핵심:** 임베딩 레이어는 **학습 중 자동으로 단어 의미를 학습**한다.

즉, "좋다"와 "훌륭하다"의 벡터가 점점 비슷해진다.

---

### (2) RNN(Recurrent Neural Network)이란?
![[image/Pasted image 20251022131356.png]]
> 데이터를 “순서대로” 읽어가며 문맥을 학습하는 네트워크

- 입력 시퀀스: 단어들의 <span style="background:#fff88f">**순서 있는 배열**</span>
- 은닉 상태(hidden state): 이전 단어의 정보를 다음으로 전달
- 주요 장점: 문맥(순서) 반영
- 한계: 긴 문장에서는 기억 손실 발생 → LSTM/GRU로 개선

---

### (3) Embedding + RNN 모델 구조

```
입력(단어 시퀀스)
   ↓
Embedding Layer (단어 → 벡터)
   ↓
RNN Layer (순서 정보 학습)
   ↓
Dense Layer (출력층, 분류)

```


In [3]:
# ==============================
# 0) 라이브러리 불러오기
# ==============================
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer        # 텍스트 → 정수 인코딩
from tensorflow.keras.preprocessing.sequence import pad_sequences # 시퀀스 길이 맞추기(패딩)
from tensorflow.keras import Sequential                           # 순차적 모델 구성 클래스
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense   # 신경망 주요 레이어

# ==============================
# 1) 미니 데이터 준비
# ==============================
# 영화 리뷰 데이터(한글 간단 문장 8개)
# 1 = 긍정(Positive), 0 = 부정(Negative)
texts = [
    "이 영화 정말 재미있다",
    "배우 연기가 훌륭하다",
    "감동적인 스토리에 눈물이 났다",
    "추천하고 싶은 영화다",
    "최악이다 다시 보고 싶지 않다",
    "지루하고 시간 낭비였다",
    "스토리가 엉성하고 별로다",
    "다시는 보고 싶지 않다"
]
labels = [1, 1, 1, 1, 0, 0, 0, 0]  # 감성 레이블

In [4]:
#  토크나이징 & 패딩
vocab_size = 1000 # 최대 단어 수 (단어사전 크기)
maxlen = 8 # 문장의 최대길이

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")  # OOV : out of vocab, 사전의 없는 단어는 oov로 대체됨
tokenizer.fit_on_texts(texts)

# 정수 시퀀스로 변경
# 가장 빈도가 높은 순서대로 숫자 배정 -> 데이터 순서대로 매핑됨
seqs = tokenizer.texts_to_sequences(texts) 

X = pad_sequences(seqs, maxlen=maxlen, padding='post', truncating='post')
y = np.array(labels)

print("샘플 시퀀스 예시:\n", X[:2])
print("단어 인덱스:", tokenizer.word_index)

샘플 시퀀스 예시:
 [[ 5  6  7  8  0  0  0  0]
 [ 9 10 11  0  0  0  0  0]]
단어 인덱스: {'<OOV>': 1, '보고': 2, '싶지': 3, '않다': 4, '이': 5, '영화': 6, '정말': 7, '재미있다': 8, '배우': 9, '연기가': 10, '훌륭하다': 11, '감동적인': 12, '스토리에': 13, '눈물이': 14, '났다': 15, '추천하고': 16, '싶은': 17, '영화다': 18, '최악이다': 19, '다시': 20, '지루하고': 21, '시간': 22, '낭비였다': 23, '스토리가': 24, '엉성하고': 25, '별로다': 26, '다시는': 27}


> padding 모든 문장의 길이를 맞추는 구문  
> [5, 6, 7, 8, 0, 0]  
> [9, 10, 11, 0, 0]  
> truncating 문장이 너무 긴 경우에는 최대 8글자였기 때문에 8자로 자름  

In [5]:

# 모델 구성 / 컴파일
# embedding과 rnn

embedding_dim = 16 # 단어를 16차원으로 임베딩
# 임베딩 [1,2,3] -> [[0.21, -0.17, 0.56..]]

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen),
    SimpleRNN(units=32),
    Dense(1, activation= "sigmoid") # 0 과 1사이의 확률값으로 산출하는 1개의 뉴런
    
])

c:\Users\shlim\miniconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [8]:
# 모델 컴파일
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=["accuracy"])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
# 모델학습
history = model.fit(X,y, epochs=20, batch_size=2, verbose=1)


Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7500 - loss: 0.6839
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8750 - loss: 0.6583 
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8750 - loss: 0.6195 
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.5810 
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.5336 
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.4744 
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.3972 
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.3082 
Epoch 9/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.2122 
Epoch 10/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.1308 
Epoch 11/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0742 
Epoch 12/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0450 
Ep

In [10]:
# 테스트
test_texts = ["정말 감동적이고 훌륭한 영화", "별로야 지루했어"]
test_seq = tokenizer.texts_to_sequences(test_texts)
test_pad = pad_sequences(test_seq, maxlen=maxlen, padding='post')
pred = (model.predict(test_pad) > 0.5).astype(int).ravel()

print("\n🔹 예측 결과:")
for t, p in zip(test_texts, pred):
    print(f"{t} → {'긍정😀' if p==1 else '부정😞'}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step

🔹 예측 결과:
정말 감동적이고 훌륭한 영화 → 긍정😀
별로야 지루했어 → 부정😞
